# COVIDvu <img src='resources/UN-flag.png' align='right'>

COVID-19 view various charts.

---
## Runtime prerequisites

In [ ]:
%%capture --no-stderr requirementsOutput

displayRequirementsOutput = False

%pip install -r requirements.txt
from covidvu.utils import autoReloadCode; autoReloadCode()

In [ ]:
if displayRequirementsOutput:
    requirementsOutput.show()

---
## Pull latest datasets from the JH CSSE GitHub repository

This function is `crontab` ready; it can be called from here, the command line, or as a system job.

In [ ]:
%%bash
./refreshdata local

---
## Predict cases

In [ ]:
COUNTRY_NAME = 'Iran'

In [ ]:
from covidvu.utils import autoReloadCode; autoReloadCode()
from covidvu.predict import predictLogisticGrowth
import covidvu.visualize as vis
import matplotlib.pyplot as plt
import pymc3 as pm

This may take ~30 mins to complete

In [ ]:
countryTS, predictionsMeanTS, predictionsPercentilesTS, countryName, trace, countryTSClean = predictLogisticGrowth(
    countryName=COUNTRY_NAME,
    nTune=100,
    nSamples=500,
    init='map',
)

In [ ]:
pm.plots.traceplot(trace);

In [ ]:
nSamples = 500
nDaysPredict = len(countryTSClean)

tStartPredict = 0
tPredict = np.arange(tStartPredict, tStartPredict + len(countryTSClean))
predictions = np.zeros((nDaysPredict, nSamples))

for i in range(nSamples):
    carryingCap = 10 ** trace['logCarryingCapacity'][i]

    predictions[:, i] = carryingCap / (
            1 + np.exp(-1.0 * trace['growthRate'][i] * (tPredict - trace['midPoint'][i])))

In [ ]:
import pandas as pd
def castAsTS(predictions, countryTSClean, nDaysPredict, tStartPredict=-1):
    predictionsTS = pd.Series(
            index = pd.date_range(
                                    start = countryTSClean.index[tStartPredict] + pd.Timedelta(1, 'D'),
                                    end   = countryTSClean.index[tStartPredict] + pd.Timedelta(nDaysPredict, 'D')
                                 ),
            data  = predictions,
        )
    return predictionsTS

In [ ]:
fig, ax = vis.plot()
countryTS.plot(ax=ax, color='black', label='Data')
predictionsMeanTS.plot(ax=ax, linestyle='-.', color='black', label='Mean Prediction')
countryTSClean.plot(ax=ax, marker='o', color='green', label='Data clean')
ax.fill_between(predictionsMeanTS.index,
                predictionsPercentilesTS[0][0],
                predictionsPercentilesTS[0][1],
                color='red',
                alpha=0.1, 
                label = r"95% CI");

ax.fill_between(predictionsMeanTS.index,
                predictionsPercentilesTS[1][0],
                predictionsPercentilesTS[1][1],
                color='red',
                alpha=0.5, 
                label = r"50% CI");

for i in range(nSamples):
    ts = castAsTS(predictions[:, i], countryTSClean, len(countryTSClean), tStartPredict=tStartPredict)
    ts.plot(ax=ax, color='black', alpha=0.01)
    
    
ax.set_ylabel('Total confirmed cases')
ax.set_title(COUNTRY_NAME)
#ax.legend(loc='upper left')
#plt.savefig(f'./figures/{COUNTRY_NAME}_prediction.png', bbox_inches='tight')

In [ ]:
import numpy as np

In [ ]:
fig, ax = vis.plot()
for i in range(nSamples):
    ts = castAsTS(predictions[:, i], countryTSClean, len(countryTSClean), tStartPredict=tStartPredict)
    np.log(ts).plot(ax=ax, color='black', alpha=0.01)
np.log(countryTSClean).plot(ax=ax, marker='o', color='green', label='Data clean')


---
&#169; the COVIDvu Contributors.  All rights reserved.